<a href="https://colab.research.google.com/github/ShreyaShinde25/Abalone-Fish-Age-Prediction-with-KNN/blob/main/Q%26AwithRAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install tiktoken
!pip install faiss-gpu
!pip install langchain_experimental
!pip install "langchain[docarray]"
!pip install langchain_community
!pip install pypdf
!pip install -U langchain-openai

# New Section

In [ ]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.vectorstores import DocArrayInMemorySearch
from IPython.display import display, Markdown
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, HuggingFaceInstructEmbeddings
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain.indexes import VectorstoreIndexCreator
from langchain_experimental.agents.agent_toolkits.csv.base import create_csv_agent
from langchain.agents.agent_types import AgentType
from langchain.memory import ConversationBufferMemory
import tiktoken
from langchain.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [ ]:
import os

# Prompt the user for their OpenAI API key
api_key = input("Please enter your OpenAI API key: ")

# Set the API key as an environment variable
os.environ["OPENAI_API_KEY"] = api_key

# Optionally, check that the environment variable was set correctly
print("OPENAI_API_KEY has been set!")


Please enter your OpenAI API key: sk-proj-zA6OQdWo5ROuRFYcGFcGT3BlbkFJA1WyWaXEP4qpfUsRouCN
OPENAI_API_KEY has been set!


In [ ]:
llm = ChatOpenAI(temperature=0.7, model_name="gpt-4")

In [ ]:
pdf_file_path = 'IntroductionToAlgo.pdf'
loader = PyPDFLoader(pdf_file_path)
pages = loader.load()

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=250)
chunks = text_splitter.split_documents(pages)

In [ ]:
# Create vector store
embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [ ]:
PROMPT_TEMPLATE = """
Answer the question based only on the following context:

{context}

---

Please specify the source ,page and chapter number when answering. Question: {question}
"""
prompt_template = ChatPromptTemplate.from_template(PROMPT_TEMPLATE)

In [ ]:
query_text = "What is Quicksort?"

In [ ]:
# results = vectorstore.similarity_search(query_text,k=3)

In [ ]:
results=retriever.invoke(query_text)

In [ ]:
context_text = "\n\n---\n\n".join(
    [f"START_SOURCE_{i} {doc.page_content} END_SOURCE_{i}\nSource: {doc.metadata['source']}, Page: {doc.metadata['page']}" for i, doc in enumerate(results)]
)

In [ ]:
prompt = prompt_template.format(context=context_text, question=query_text)
# print(prompt)

In [ ]:
response_text = llm.predict(prompt)

In [ ]:
response_text

'Quicksort is an algorithm used for sorting that applies the divide-and-conquer paradigm. It works by partitioning an array into two subarrays, such that each element of the first subarray is less than or equal to a chosen element, which is in turn less than or equal to each element of the second subarray. It then sorts the two subarrays by recursive calls to quicksort. Quicksort has a worst-case running time of n2 on an input array of n numbers, but is often the best practical choice for sorting because it is remarkably efficient on average, with an expected running time of n lgn. It also has the advantage of sorting in place and works well even in virtual-memory environments (IntroductionToAlgo.pdf, Page: 190, Chapter 7).'

In [ ]:
response_text

'Quicksort is an algorithm used for sorting that applies the divide-and-conquer paradigm. It partitions an array into two subarrays such that each element of the first subarray is less than or equal to a chosen element, which is, in turn, less than or equal to each element of the second subarray. The two subarrays are then sorted by recursive calls to quicksort. Quicksort has a worst-case running time of n^2, but it is often the best practical choice for sorting due to its efficiency on average, with an expected running time of n log(n). It also has the advantage of sorting in place and works well in virtual-memory environments (Source: IntroductionToAlgo.pdf, Page: 190, Chapter 7).'

In [ ]:
response_text

'Quicksort is an algorithm that applies the divide-and-conquer paradigm for sorting. It has a worst-case running time of ‚.n2/ on an input array of n numbers. Despite this slow worst-case running time, quicksort is often the best practical choice for sorting because it is remarkably efficient on average: its expected running time is ‚.n lgn/, and the constant factors hidden in the ‚.n lgn/ notation are quite small. It also has the advantage of sorting in place and works well even in virtual-memory environments (Source: IntroductionToAlgo.pdf, Page: 190, Chapter 7).'